# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import helper
import azureml.core
from azureml.data.datapath import DataPath
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.run import Run
from azureml.core.model import Model


from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Load workspace from config file present at .\config.json.
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# Choose a name for experiment
experiment_name = 'Titanic_hyperdrive'
project_folder = './titanic-project-hyperdrive'

experiment=Experiment(ws, experiment_name)
experiment

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Titanic_dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        datastore = ws.get_default_datastore()
        datastore.upload(src_dir='data', target_path='data')
        train_data = datastore.path('data/train_modified.csv')
        
        dataset = Dataset.Tabular.from_delimited_files(train_data, separator=';')        
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description="This is the complete dataset for the capstone project.")
        
dataset_filtered = dataset.keep_columns(["Survived","Pclass","Sex","SibSp","Parch","Fare","Embarked","Age"])
dataset_filtered = dataset_filtered.register(workspace=ws,
                           name=key+"_filtered",
                           description="This is the filtered dataset for the capstone project " \
                           "with only those features relevant for training.")

df = dataset_filtered.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

num_nodes = 5

amlcompute_cluster_name = "ComputeClusterCapstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=num_nodes)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here>

In [ ]:
#TODO: Submit your experiment
remote_run = 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run, best_model = remote_run.get_output()
print(best_run)
print(best_model)

In [ ]:
#TODO: Save the best model
model_dir = "Hyperdrive_model"  # Local folder where the model will be stored temporarily
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

best_run.download_file("outputs/model.pkl", model_dir + "/model.pkl")

In [ ]:
import sklearn
from azureml.core.resource_configuration import ResourceConfiguration
# Register the model
model_name = "Titanic_Hyperdrive_model"
best_model = Model.register(workspace=ws,
                            model_name=model_name,
                            model_path=model_dir + "/model.pkl",
                            model_framework=Model.Framework.SCIKITLEARN,
                            model_framework_version=sklearn.__version__,
                            sample_input_dataset=dataset_filtered,
                            resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                            description="Hyperdrive model to predict Titanic survivors.",
                            tags=None
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

[Source1](https://docs.microsoft.com/de-de/python/api/overview/azure/ml/?view=azure-ml-py)
[Source 2](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb)

TODO: In the cell below, create an inference config and deploy the model as a web service.

In [61]:
env = Environment.get(ws, "AzureML-AutoML")
print(env)
for pip_package in ["skikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

# Update scoring script
datastore.upload_files(['./score.py'], overwrite=True)

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   source_directory=".",
                                   environment=env)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                       enable_app_insights=True,
                                                       auth_enabled=False)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "titanic-webservice",
                       models = [best_model],
                       inference_config = inference_config,
                       deployment_config = deployment_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Environment(Name: AzureML-AutoML,
Version: 115)
Uploading an estimated of 1 files
Uploading ./score.py
Uploaded ./score.py, 1 files out of an estimated total of 1
Uploaded 1 files
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-03 09:25:39+00:00 Creating Container Registry if not exists.
2022-06-03 09:25:39+00:00 Registering the environment.
2022-06-03 09:25:40+00:00 Use the existing image.
2022-06-03 09:25:40+00:00 Generating deployment configuration.
2022-06-03 09:25:41+00:00 Submitting deployment to compute..
2022-06-03 09:26:14+00:00 Checking the status of deployment titanic-webservice..
2022-06-03 09:28:15+00:00 Checking the status of inference endpoint titanic-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2022-06-03T09:27:30,803388000+00:00 - gunicorn/run 
2022-06-03T09:27:30,806797400+00:00 | gu

# Prediction
Predict values for `Survived` for the Kaggle competition.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [25]:
service = Webservice(workspace=ws, name="titanic-webservice")
print(service)

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-197423', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-197423'), name=titanic-webservice, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://8b4a0227-956e-4b86-8ac9-811bab24acc9.southcentralus.azurecontainer.io/score, tags=None, properties={'hasInferenceSchema': 'False', 'hasHttps': 'False'}, created_by={'userObjectId': '016836ad-a3f6-4af5-a9a5-b308069434a4', 'userPuId': '100320020175254D', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/660b3398-b80e-49d2-bc5b-ac1dc93b5254/', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 197423', 'upn': 'odl_user_197423@udacitylabs.onmicrosoft.com'})


In [ ]:
import json
def predict_from_df(df):
    df_json = df.to_json(orient='records')

    input_payload = json.dumps({
        'data': json.loads(df_json),
        'method': 'predict'
    }, indent=2)
    #print(input_payload)

    output = service.run(input_payload)
    #print("Response:\n",output)
    return output

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Titanic_dataset_test"

if key in ws.datasets.keys(): 
    print("Found", key)
    found = True
    dataset_predict = ws.datasets[key] 

if not found:
    print("Did not find", key)
    print("Create the dataset")
    # Create AML Dataset and register it into Workspace
    datastore = ws.get_default_datastore()
    datastore.upload(src_dir='data', target_path='data')
    predict_data = datastore.path('data/test_modified.csv')

    dataset_predict = Dataset.Tabular.from_delimited_files(predict_data, separator=';')        
    dataset_predict = dataset_predict.register(workspace=ws,
                               name=key,
                               description="This is the test dataset for the capstone project.")

found = False
key_filtered = key+"_filtered"

if key_filtered in ws.datasets.keys(): 
    print("Found", key_filtered)
    found = True
    dataset_predict_filtered = ws.datasets[key_filtered] 

if not found:
    print("Did not find", key)
    print("Create the dataset")
    dataset_predict_filtered = dataset_predict.keep_columns(["Pclass","Sex","SibSp","Parch","Fare","Embarked","Age"])
    dataset_predict_filtered = dataset_predict_filtered.register(workspace=ws,
                               name=key_filtered,
                               description="This is the filtered test dataset for the capstone project " \
                               "with only those features relevant for prediction."
                               )
    
print(dataset_predict_filtered[:2])
dataframe_predict_filtered = dataset_predict_filtered.to_pandas_dataframe()

y_pred = predict_from_df(dataframe_predict_filtered)

dataset_predict_output = dataset_predict.keep_columns(["PassengerId"])
dataframe_predict_output = dataset_predict_output.to_pandas_dataframe()
dataframe_predict_output = pd.concat([dataframe_predict_output, pd.DataFrame(y_pred, columns=["Survived"])], axis=1)


datastore = ws.get_default_datastore()
dataframe_predict_output.to_csv("dataset_test_predictions_hyperdrive.csv", index=False)
dataset_predict = Dataset.Tabular.register_pandas_dataframe(dataframe_predict_output,
                                                            target=datastore,
                                                            name=key+"_filtered_predict",
                                                            description="These are the preditions for the test dataset."
                                                            )

In [ ]:
print(predict_from_df(dataframe_predict_filtered[0:10]))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
!python logs.py

In [ ]:
# Delete the webservice
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

